# Compare GPT-2 with LLaMA 7B results

In [1]:
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import math
import scipy
import random
from tqdm import tqdm

In [2]:
relations = ["P19", "P20", "P27", "P101", "P495", "P740", "P1376"]

data_folder = "/cephyr/users/lovhag/Alvis/projects/rome/data/RQ1"

In [39]:
model_names = ["gpt2_xl", "llama7B"]

data = pd.DataFrame()

for model_name in model_names:
    for relation in relations:
        tmp_data = pd.read_csv(os.path.join(data_folder, model_name, f"{relation}.csv"))
        tmp_data["relation"] = relation
        tmp_data["model"] = model_name
        data = data.append(tmp_data, ignore_index=True)
        
data

,sample_ix,lama_ix,subject,lama_template,sample_template,lama_answer,sample_answer,correct_answer,lama_correct,sample_correct,lama_te,sample_te,is_consistent,pairwise_sim,subj_len,token_ix,pred_type,relation,model
0,1,0,Allan Peiper,{} was born in,{} is originally from,New,the,Alexandra,False,False,0.023766,0.021967,False,0.998697,4,0,any,P19,gpt2_xl
1,1,0,Allan Peiper,{} was born in,{} is originally from,London,Canada,Alexandra,False,False,0.017587,0.003975,False,0.995587,4,2,candidate,P19,gpt2_xl
2,2,0,Allan Peiper,{} was born in,{} was originally from,New,the,Alexandra,False,False,0.023766,0.018423,False,0.998855,4,0,any,P19,gpt2_xl
3,2,0,Allan Peiper,{} was born in,{} was originally from,London,Canada,Alexandra,False,False,0.017587,0.003594,False,0.996345,4,2,candidate,P19,gpt2_xl
4,3,0,Allan Peiper,{} was born in,{} is native to,New,the,Alexandra,False,False,0.023766,0.022872,False,0.994816,4,0,any,P19,gpt2_xl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121936,447,444,Topeka,{} is the capital of,"{}, the capital city of",Kansas,Kansas,Kansas,True,True,0.525278,0.894565,True,0.971347,3,0,candidate,P1376,llama7B
121937,448,444,Topeka,{} is the capital of,"{}, that is the capital of",Kansas,Kansas,Kansas,True,True,0.525278,0.732707,True,0.984375,3,0,any,P1376,llama7B
121938,448,444,Topeka,{} is the capital of,"{}, that is the capital of",Kansas,Kansas,Kansas,True,True,0.525278,0.732707,True,0.984375,3,0,candidate,P1376,llama7B
121939,449,444,Topeka,{} is the capital of,"{}, that is the capital city of",Kansas,Kansas,Kansas,True,True,0.525278,0.813442,True,0.963304,3,0,any,P1376,llama7B


## Overlapping results before any data filtering

In [40]:
data[data[["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"]].duplicated(keep=False)].sort_values(by=["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"])

,sample_ix,lama_ix,subject,lama_template,sample_template,lama_answer,sample_answer,correct_answer,lama_correct,sample_correct,lama_te,sample_te,is_consistent,pairwise_sim,subj_len,token_ix,pred_type,relation,model
32147,3284,3283,A Thousand Plateaus,{} works in the field of,{} specializes in,philosophy,art,philosophy,True,False,0.008895,0.000086,False,0.967288,4,2,candidate,P101,gpt2_xl
87465,1520,1519,A Thousand Plateaus,{} works in the field of,{} specializes in,philosophy,art,philosophy,True,False,0.048340,0.000043,False,0.979823,7,1,candidate,P101,llama7B
32157,3289,3283,A Thousand Plateaus,{} works in the field of,{} works in the area of,philosophy,philosophy,philosophy,True,True,0.008895,0.006392,True,0.959181,4,2,candidate,P101,gpt2_xl
87475,1525,1519,A Thousand Plateaus,{} works in the field of,{} works in the area of,philosophy,philosophy,philosophy,True,True,0.048340,0.043042,True,0.998574,7,1,candidate,P101,llama7B
32151,3286,3283,A Thousand Plateaus,{} works in the field of,{}'s domain of work is,philosophy,art,philosophy,True,False,0.008895,-0.000274,False,0.886277,4,2,candidate,P101,gpt2_xl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111199,1854,1848,Trenitalia,{} was founded in,"{}, that was created in",Italy,Italy,Rome,False,False,0.027545,0.013227,True,0.977845,4,2,candidate,P740,llama7B
51796,2617,2604,Trenitalia,{} was founded in,"{}, that was formed in",Italy,Italy,Rome,False,False,0.012077,0.012718,True,0.991316,4,2,candidate,P740,gpt2_xl
111213,1861,1848,Trenitalia,{} was founded in,"{}, that was formed in",Italy,Italy,Rome,False,False,0.027545,0.014946,True,0.990039,4,2,candidate,P740,llama7B
51776,2607,2604,Trenitalia,{} was founded in,"{}, that was started in",Italy,Italy,Rome,False,False,0.012077,0.013491,True,0.997087,4,2,candidate,P740,gpt2_xl


In [41]:
len(data[data[["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"]].duplicated()])

8646

In [42]:
data[data[["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"]].duplicated(keep=False)].groupby("model").mean()

,sample_ix,lama_ix,lama_correct,sample_correct,lama_te,sample_te,is_consistent,pairwise_sim,subj_len,token_ix
model,,,,,,,,,,
gpt2_xl,3861.226694,3856.557715,0.742077,0.72762,0.163002,0.138434,0.907819,0.987150,4.688064,1.358663
llama7B,3044.367453,3039.698473,0.742077,0.72762,0.255807,0.233380,0.907819,0.984224,4.749133,0.863983


In [43]:
data[data[["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"]].duplicated(keep=False)].groupby(["model", "relation"]).relation.count()

model    relation
gpt2_xl  P101         352
         P1376        587
         P19          696
         P20          785
         P27         4056
         P495        1752
         P740         418
llama7B  P101         352
         P1376        587
         P19          696
         P20          785
         P27         4056
         P495        1752
         P740         418
Name: relation, dtype: int64

## Overlapping results after filtering

In [44]:
# remove forbidden preds
forbidden_predictions = ["a", "the", "collaboration", "response", "public", '"', "order", "partnership", "honor", "AD", "open", "H", "age", "creating", "disgrace", "her", "his", "in", "left", "not", "providing", "tragedy", "which", "whom", "I", "non", "cy", "Se", "Tw", "ac", "ps", "Te", "ne", "spe", "B", "compar", "bi", "Pe"]
print("Removing the forbidden predictions for the LAMA paraphrases...")
print(data[data.lama_answer.isin(forbidden_predictions)].lama_answer.value_counts())

forbidden_mask = (data.lama_answer.isin(forbidden_predictions))# | (data.sample_answer.isin(forbidden_predictions))
data = data[~(forbidden_mask)].reset_index(drop=True)
data = data.dropna() # drop values for which LLaMA produced ""

# remove samples for which TE is low
te_thresh = 0.1
data = data[(data.sample_te>te_thresh) & (data.lama_te>te_thresh)]

data

Removing the forbidden predictions for the LAMA paraphrases...
the              31574
a                 2877
public             300
response            64
collaboration       32
"                   24
partnership         16
honor               16
order               16
Pe                  13
open                12
his                  7
AD                   7
non                  6
Se                   6
spe                  6
B                    6
cy                   6
ne                   6
compar               6
Te                   6
I                    6
ac                   6
ps                   6
Tw                   6
bi                   6
H                    5
Name: lama_answer, dtype: int64


,sample_ix,lama_ix,subject,lama_template,sample_template,lama_answer,sample_answer,correct_answer,lama_correct,sample_correct,lama_te,sample_te,is_consistent,pairwise_sim,subj_len,token_ix,pred_type,relation,model
372,281,280,Parviz Davoodi,{} was born in,{} is originally from,Iran,Iran,Tehran,False,False,0.137497,0.311235,True,0.993288,6,0,any,P19,gpt2_xl
374,282,280,Parviz Davoodi,{} was born in,{} was originally from,Iran,Iran,Tehran,False,False,0.137497,0.232244,True,0.995910,6,0,any,P19,gpt2_xl
376,283,280,Parviz Davoodi,{} was born in,{} is native to,Iran,Iran,Tehran,False,False,0.137497,0.211879,True,0.986838,6,0,any,P19,gpt2_xl
378,284,280,Parviz Davoodi,{} was born in,{} was native to,Iran,Iran,Tehran,False,False,0.137497,0.193289,True,0.992252,6,0,any,P19,gpt2_xl
452,352,350,Ludwig Marcuse,{} was born in,{} was originally from,Vienna,Germany,Berlin,False,False,0.144169,0.100243,False,0.994134,5,0,any,P19,gpt2_xl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86895,447,444,Topeka,{} is the capital of,"{}, the capital city of",Kansas,Kansas,Kansas,True,True,0.525278,0.894565,True,0.971347,3,0,candidate,P1376,llama7B
86896,448,444,Topeka,{} is the capital of,"{}, that is the capital of",Kansas,Kansas,Kansas,True,True,0.525278,0.732707,True,0.984375,3,0,any,P1376,llama7B
86897,448,444,Topeka,{} is the capital of,"{}, that is the capital of",Kansas,Kansas,Kansas,True,True,0.525278,0.732707,True,0.984375,3,0,candidate,P1376,llama7B
86898,449,444,Topeka,{} is the capital of,"{}, that is the capital city of",Kansas,Kansas,Kansas,True,True,0.525278,0.813442,True,0.963304,3,0,any,P1376,llama7B


In [45]:
data[data[["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"]].duplicated(keep=False)].sort_values(by=["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"])

,sample_ix,lama_ix,subject,lama_template,sample_template,lama_answer,sample_answer,correct_answer,lama_correct,sample_correct,lama_te,sample_te,is_consistent,pairwise_sim,subj_len,token_ix,pred_type,relation,model
19754,2799,2793,David Bohm,{} works in the field of,{} works in the area of,quantum,quantum,physics,False,False,0.191636,0.157499,True,0.987774,3,0,any,P101,gpt2_xl
52761,1280,1274,David Bohm,{} works in the field of,{} works in the area of,quantum,quantum,physics,False,False,0.335640,0.301773,True,0.994503,3,0,any,P101,llama7B
20037,3079,3073,Edinburgh Mathematical Society,{} works in the field of,{} works in the area of,mathematics,mathematics,mathematics,True,True,0.219180,0.129015,True,0.997691,5,0,candidate,P101,gpt2_xl
52990,1427,1421,Edinburgh Mathematical Society,{} works in the field of,{} works in the area of,mathematics,mathematics,mathematics,True,True,0.316145,0.145325,True,0.993286,4,0,candidate,P101,llama7B
19040,2071,2065,Edward Teller,{} works in the field of,{} works in the area of,nuclear,nuclear,physics,False,False,0.282333,0.222657,True,0.997018,3,0,any,P101,gpt2_xl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73833,411,406,The Coca-Cola Company,{} was founded in,{} was started in,Atlanta,Atlanta,Atlanta,True,True,0.155066,0.105983,True,0.988513,7,1,candidate,P740,llama7B
23812,597,588,The Coca-Cola Company,{} was founded in,"{}, that originated in",Atlanta,Atlanta,Atlanta,True,True,0.136602,0.227166,True,0.980291,5,2,candidate,P740,gpt2_xl
73841,415,406,The Coca-Cola Company,{} was founded in,"{}, that originated in",Atlanta,Atlanta,Atlanta,True,True,0.155066,0.400476,True,0.984087,7,1,candidate,P740,llama7B
23800,591,588,The Coca-Cola Company,{} was founded in,"{}, that was started in",Atlanta,Atlanta,Atlanta,True,True,0.136602,0.210853,True,0.986753,5,2,candidate,P740,gpt2_xl


In [46]:
len(data[data[["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"]].duplicated()])

2593

In [47]:
data[data[["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"]].duplicated(keep=False)].groupby(["model", "relation"]).relation.count()

model    relation
gpt2_xl  P101          13
         P1376        464
         P19           20
         P20            1
         P27         2080
         P495           6
         P740           9
llama7B  P101          13
         P1376        464
         P19           20
         P20            1
         P27         2080
         P495           6
         P740           9
Name: relation, dtype: int64

In [48]:
data[data[["relation", "subject", "lama_template", "sample_template", "lama_answer", "sample_answer", "pred_type"]].duplicated(keep=False)].groupby("model").mean()

,sample_ix,lama_ix,lama_correct,sample_correct,lama_te,sample_te,is_consistent,pairwise_sim,subj_len,token_ix
model,,,,,,,,,,
gpt2_xl,2732.086772,2728.563440,0.977632,0.974547,0.421764,0.374754,0.996143,0.991187,4.860779,0.195141
llama7B,2102.590436,2099.067104,0.977632,0.974547,0.499739,0.519220,0.996143,0.990647,4.860779,0.165060


Compare to the stats for each model, samples not necessarily overlapping.

In [49]:
data.groupby("model").mean()

,sample_ix,lama_ix,lama_correct,sample_correct,lama_te,sample_te,is_consistent,pairwise_sim,subj_len,token_ix
model,,,,,,,,,,
gpt2_xl,2821.651766,2817.963771,0.753244,0.716835,0.400635,0.365596,0.929885,0.990137,4.591565,0.186734
llama7B,2520.667303,2516.484556,0.923621,0.875211,0.402166,0.405144,0.931821,0.984017,4.702572,0.334831
